In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('merged_filtered_data.csv')
data.head()

,Unnamed: 0,Date,Temperature,Humidity,Hours Since Previous Failure,Failure,Operator_Operator1,Operator_Operator2,Operator_Operator3,Operator_Operator4,...,m6,m7,m8,m9,m10,m11,m12,m13,m14,m15
0,0,2016-01-01 00:00:00,67,82,90,0,1,0,0,0,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,1,2016-01-01 01:00:00,68,77,91,0,1,0,0,0,...,368.501342,596.502482,312.001060,509.502047,172.500362,251.500757,411.501557,773.003365,409.501547,424.001620
2,2,2016-01-01 02:00:00,64,76,92,0,1,0,0,0,...,619.338409,1047.009163,835.342762,773.339095,206.000800,810.009813,648.671658,754.335266,779.007190,845.674774
3,3,2016-01-01 03:00:00,63,80,93,0,1,0,0,0,...,817.760751,1073.007782,862.758031,908.009035,528.011870,1046.265628,845.010615,691.249241,918.260266,812.004902
4,4,2016-01-01 04:00:00,65,81,94,0,1,0,0,0,...,700.801584,1146.610642,1037.416904,933.008728,732.221747,997.409572,1039.820179,876.210490,1036.015278,751.000262


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 34 columns):
Unnamed: 0                      8784 non-null int64
Date                            8784 non-null object
Temperature                     8784 non-null int64
Humidity                        8784 non-null int64
Hours Since Previous Failure    8784 non-null int64
Failure                         8784 non-null int64
Operator_Operator1              8784 non-null int64
Operator_Operator2              8784 non-null int64
Operator_Operator3              8784 non-null int64
Operator_Operator4              8784 non-null int64
Operator_Operator5              8784 non-null int64
Operator_Operator6              8784 non-null int64
Operator_Operator7              8784 non-null int64
Operator_Operator8              8784 non-null int64
Measure2_0                      8784 non-null int64
Measure2_1                      8784 non-null int64
Measure2_2                      8784 non-null int64
Measure2

In [5]:
data = data.drop(['Unnamed: 0', 'Date'], axis=1)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 32 columns):
Temperature                     8784 non-null int64
Humidity                        8784 non-null int64
Hours Since Previous Failure    8784 non-null int64
Failure                         8784 non-null int64
Operator_Operator1              8784 non-null int64
Operator_Operator2              8784 non-null int64
Operator_Operator3              8784 non-null int64
Operator_Operator4              8784 non-null int64
Operator_Operator5              8784 non-null int64
Operator_Operator6              8784 non-null int64
Operator_Operator7              8784 non-null int64
Operator_Operator8              8784 non-null int64
Measure2_0                      8784 non-null int64
Measure2_1                      8784 non-null int64
Measure2_2                      8784 non-null int64
Measure2_3                      8784 non-null int64
Measure3_0                      8784 non-null int64
Measure3_

In [10]:
features = data.drop('Failure', axis=1)
target = data.Failure

In [15]:
#upsample minority
from imblearn.over_sampling import SMOTENC

features_rs, target_rs = SMOTENC(categorical_features = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],sampling_strategy='auto',n_jobs=-1).fit_sample(features, target)
print(pd.Series(target_rs).value_counts()) #Preview synthetic sample class distribution

1    8703
0    8703
dtype: int64


Exception: Data must be 1-dimensional

In [18]:
import numpy as np
np.count_nonzero(features_rs)

320619

In [19]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
scaler = StandardScaler()
scaled_df = scaler.fit_transform(features_rs)
##### set train, test
X_train, X_test, y_train, y_test = train_test_split(scaled_df, target_rs, test_size=0.25)
###### create xgb instance with parameters
clf = xgb.XGBClassifier()
###### fit data
clf.fit(X_train, y_train)
###### make predictions
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
###### run classification report and confusion matrix
gbt_confusion_matrix = confusion_matrix(y_test, val_preds)
gbt_confusion_matrix
gbt_classification_report = classification_report(y_test, val_preds)
print(gbt_classification_report)
###### cross validation score
print('Mean XGB Cross-Val Score (k=5):')
print(cross_val_score(clf, scaled_df, target_rs, cv=5).mean())

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2219
           1       1.00      1.00      1.00      2133

    accuracy                           1.00      4352
   macro avg       1.00      1.00      1.00      4352
weighted avg       1.00      1.00      1.00      4352

Mean XGB Cross-Val Score (k=5):
0.9876480685561871
